In [1]:
# mandatory imports
from xcube.core.store import find_data_store_extensions
from xcube.core.store import get_data_store_params_schema
from xcube.core.store import new_data_store
from xcube.core.select import select_label_subset

# Utilities for notebook visualization
import shapely.geometry
import IPython.display
import pandas as pd
from IPython.display import JSON
import matplotlib.pyplot as plt
import math

In [2]:
sh_store = new_data_store('sentinelhub', num_retries=400)

DataStoreError: Unknown data store "sentinelhub" (may be due to missing xcube plugin)

In [ ]:
sh_store.get_open_data_params_schema('S2L2A')

In [ ]:
mc_def = pd.read_csv("../3_eventcoord4minicube.csv", delimiter="\t")
mc_def

In [ ]:
r_earth = 6378137
spatial_res_m = 20
spatial_res = 360 * spatial_res_m / (2 * math.pi * r_earth)
spatial_res

In [ ]:
spatial_res = 0.00018
image_size = 256

In [ ]:
regions = []
for row in mc_def.iterrows():
    name, (center_lon, center_lat, _) = row
    spatial_size = spatial_res * image_size / 2
    bbox = (center_lon - spatial_size,
            center_lat - spatial_size,
            center_lon + spatial_size,
            center_lat + spatial_size)
    regions.append((name, (center_lon, center_lat), bbox))

In [ ]:
polygon = shapely.geometry.MultiPolygon([shapely.geometry.box(*bbox) 
                                        for _, _, bbox in regions])
IPython.display.GeoJSON(polygon.__geo_interface__)

In [ ]:
s3_store = new_data_store("s3", root="deepextremes", storage_options=dict(anon=False))

In [ ]:
for name, center, bbox in regions:
    mc = sh_store.open_data('S2L2A', 
                            variable_names=['B02', 'B03', 'B04', 'B06', 'B8A',  'B11', 'SCL'], 
                            bbox=bbox, 
                            spatial_res=spatial_res, 
                            time_range=('2016-01-01','2022-08-01'), 
                            time_period=None)
    out_path = f"s2_l2a_{name}_x{center[0]}_y{center[1]}.zarr"
    display(f"Writing {out_path}...")
    mc.to_zarr(f"s2_l2a_{name}_x{center[0]}_y{center[1]}.zarr", mode="w")
    # s3_store.write_data(mc, f"s2_l2a_{name}_x{center[0]}_y{center[1]}.zarr")
    display(f"Done.")    